In [ ]:
import pandas as pd
import re
import os

In [ ]:

search_dir = 'Coli_2_files'
search_dir_with_extra_slash = search_dir + '/'


In [ ]:
def deltermini_label(dataframe):

    string = dataframe

    # Use regular expression to match "sum=" and "islen=" followed by numbers
    sum_match = re.search(r'sum=(\d+-\d+:\d+-\d+)', string)
    islen_match = re.search(r'islen=(\d+)', string)

    # Extract the matched values
    sum_value = sum_match.group(1) if sum_match else None
    islen_value = islen_match.group(1) if islen_match else None

    sum_values = sum_value.split("-")  # First split by "-"
    sum_values = [value.split(":") for value in sum_values]  # Then split by ":"

    # formula
    del_termini = 1 - int(sum_values[0][0]) + int(sum_values[1][0])- int(islen_value) 

    if del_termini==0:
        label = "Complete"
    else:
        label = "Incomplete"

    return label

In [ ]:
def del_internal(string):

    sum_value = string.split("sum=")[1].split(";")[0]

    sum_string = sum_value

    # Define a regular expression pattern to match any character that is not a digit.
    pattern = r"[^0-9]+"

    # Use the re.split() method to split the string at any character that is not a digit.
    numbers = re.split(pattern, sum_string)

    # Filter out any empty strings that may have resulted from the split.
    numbers = list(filter(None, numbers))

    #formula
    del_internal = (int(numbers[1]) - int(numbers[4])) + (int(numbers[3]) - int(numbers[6]))

    del_internal

    if del_internal > 0:
        label = "insertion"
    else:
        label = "delation"

    return label

In [ ]:
def match_ter(search_dir, foldername):

    # replace 'file.csv' with the actual file name and path
    df1 = pd.read_csv(search_dir + '/'+foldername + '/genome.IS.blast.csv')
    
    blast_df = df1.copy().dropna(axis=1)

    list_label = []
    list_internallabel = []
    
    for att_str in blast_df['INFO']:
        list_label.append(deltermini_label(att_str))
        if "+" in att_str.split("sum=")[1]:
            list_internallabel.append(del_internal(att_str))
        else:
            list_internallabel.append("None")


    blast_df['del_termini'] = list_label
    blast_df['del_internal'] = list_internallabel
    
    blast_df.to_csv(search_dir + '/'+foldername + '/deltermini_out.csv') 

    #####
    return -1

In [ ]:
for entry in os.listdir(search_dir):
    foldername = entry.replace(search_dir_with_extra_slash, '')
    match_ter(search_dir, foldername=foldername)